In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install speechrecognition

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install keybert

In [ ]:
from base64 import encode
import speech_recognition as sr
import wave
import numpy as np
import os
import pandas as pd
import faiss
import nltk
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT

In [ ]:
def get_duration_int(fname):
    audio = wave.open(fname)
    frames = audio.getnframes()
    rate = audio.getframerate()
    duration = frames / float(rate)
    return int(duration)

In [ ]:
def stt(fname, duration):
    r = sr.Recognizer()
    data = sr.AudioFile(fname) #'./iphone.wav'
    iter = (duration // 60) +1
    i = 0
    doc = ''
    for k in range(iter):
        with data as source:
            audio = r.record(source, offset = i, duration = 60)
        text = r.recognize_google(audio)
        doc += text
        i+=60
    return doc

In [ ]:
# 마침표 없는 텍스트의 문장화 (문장 당 동사 1개. 동사로 시작.)
def maksen_v(doc): 
    nltk.download('averaged_perceptron_tagger')
    nltk.download('punkt')
    listtext = nltk.tokenize.word_tokenize(doc)
    tagged = nltk.tag.pos_tag(listtext)
    sent = []
    sentlist = []
    counter = 0
    for a, b in tagged:
        if b[0] == 'V':
            if counter == 0:
                counter +=1
                sent.append(a)
            else:
                sentlist.append(' '.join(sent))
                sent.clear()
                sent.append(a)
        else:
            sent.append(a)         
    return sentlist            

In [ ]:
# 마침표 없는 텍스트의 문장화 (문장연결사를 중심으로 문장 분할)
def maksen_cc(doc): 
    nltk.download('averaged_perceptron_tagger')
    nltk.download('punkt')
    listtext = nltk.tokenize.word_tokenize(doc) 
    tagged = nltk.tag.pos_tag(listtext) 
    sent = []
    sentlist = []
    counter = 0
    for a, b in tagged:
        if b == 'CC' or b == 'RB':
            if len(sent) < 7:
              sent.append(a)
              continue
            sentlist.append(' '.join(sent))
            sent.clear()
            sent.append(a)
        else:
            sent.append(a)         
    return sentlist           

In [ ]:
def semantic(sent_doc, query):
    #임베딩
    model = SentenceTransformer('distilbert-base-nli-mean-tokens')
    encoded_data = model.encode(sent_doc)
    #인덱스
    index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
    index.add_with_ids(encoded_data, np.array(range(0,len(sent_doc))))
    faiss.write_index(index, 'semantic_iphone')
    #검색
    query_vector = model.encode([query])
    k = 5
    top_k = index.search(query_vector, k)
    
    return [sent_doc[_id] for _id in top_k[1].tolist()[0]]

In [ ]:
#if __name__ == "__main__":

#<STT>
fname = "./iphone.wav"
#duration = get_duration_int(fname)
#doc = stt(fname, duration)
#print(doc)
    
dup_doc= """
            from using your iPhone is a scanner to stacking your favorite home screen widgets your are 10 helpful iPhone tips you should know did you accidentally enter a wrong digit into the calculator no problem just swipe left or right at the top of the screen to erase it bonus tip you can copy numbers by touching and holding the digits 
            and then tapping copy starting an iOS 15 when someone sends you things like web links or Apple news articles and messages you can pin the contents so it's easier to find when you need it in a messages read touch and hold the item you want to pin and then tap pin pin content will be elevated in messages search the details view of a conversation and apps that support shared with youorganize your home screen by stacking your widgets touch 
            and hold an empty space on your home screen to edit it then drag One widget on top of another of the same size you can add up to 10 widgets when creating a stacked tap done in the upper right corner when you're done and 
            the widgets will automatically rotate to show the most relevant information throughout the day you can also swipe through them if you want to switch to a different one select multiple photos at the same time and then add them to other apps with this multi-touch gestures touch and hold a photo until you feel attacked and then dragged it away a little while still holding it with a different finger tap another photo to add it to the stack 
            you can select as many photos as you like once you selected the photos switch to the supported app you want to share two while still holding the Stock drag your finger wherewant to add them and lift your finger to drop them into the app if you find yourself typing out the same phrase a lot creative text replacement to get the message across faster in settings tap General then tap keyboard and tap text replacement tap the add button in the top-right corner and enter a phrase into the phrase field then enter the text replacement you like to use into the shortcut field and tap save now whenever you type the text replacement the phrase will appear in the predictive text section of your keyboard you can quickly swipe to open the camera without even having to unlock your screen just swipe left got itdid you know that you can scan a document with a Notes app on your iPhone in a note tap the camera button and tap scan documents position the document in the viewfinder and it will scan automatically repeat the process with his men Pages as you need to scan tap save when you're done starting an iOS 15 you can use live text to interact with text using your camera just position of you find her over a document and tap the scan text button in the bottom right corner you can now interact with a text that appears for example you can tap select all then tap the screen to see the editing options again and tap copy then switch over to another app and paste the text that you just copiedin the photos app if you're looking for the first 
            photo in an album library or search results you can quickly jump to the beginning of the view by tapping the top edge of your iPhone screen tap the tab button for the view you're in at the bottom of the screen to jump back to the most recent photo you can quickly access search right on your home screen and an iOS 15 your lock screen to just wipe your finger down from the middle of the home screen or lock screen and a search field will appear type in what you're looking for to quickly launch an app find a contact get web search suggestions and more gouda mac and cheese sounds delicious with these helpful tips here on your way to discovering more of what your iPhone has to offer to learn more about how to use your iPhone subscribe to the Apple support YouTube channelgo to keep watching Wikipedia is a registered trademark of the Wikimedia Foundation Incorporated a nonprofit organization
         """

#<텍스트 문장화_ngram 이용>
n_range = (5,5)
count = CountVectorizer(ngram_range = n_range, stop_words = None).fit([dup_doc])
candidates = count.get_feature_names_out()

model = SentenceTransformer('distilbert-base-nli-mean-tokens') 
candidate_embeddings = model.encode(candidates)
  
#<semantic search를 위한 검색어 입력>
query = str(input("검색어를 입력해주세요 : "))

#<semantic search 과정_코사인유사도와 SBERT이용>
query_embedding = model.encode([query])
top_n = 5
distances = cosine_similarity(query_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[][-top_n:]]0

#<결과 출력_코사인유사도와 SBERT이용>
print('<results>')
for res in keywords:
    print('\t',res)    

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

검색어를 입력해주세요 : taking photo
<results>
	 multiple photos at the same
	 photos at the same time
	 in the viewfinder and it
	 document in the viewfinder and
	 gestures touch and hold photo


In [ ]:
#if __name__ == "__main__":

#<STT>
fname = "./iphone.wav"
#duration = get_duration_int(fname)
#dup_doc = stt(fname, duration)
#print(dup_doc)

#<시간걸리는 STT과정의 출력값>
dup_doc= """
            from using your iPhone is a scanner to stacking your favorite home screen widgets your are 10 helpful iPhone tips you should know did you accidentally enter a wrong digit into the calculator no problem just swipe left or right at the top of the screen to erase it bonus tip you can copy numbers by touching and holding the digits 
            and then tapping copy starting an iOS 15 when someone sends you things like web links or Apple news articles and messages you can pin the contents so it's easier to find when you need it in a messages read touch and hold the item you want to pin and then tap pin pin content will be elevated in messages search the details view of a conversation and apps that support shared with youorganize your home screen by stacking your widgets touch 
            and hold an empty space on your home screen to edit it then drag One widget on top of another of the same size you can add up to 10 widgets when creating a stacked tap done in the upper right corner when you're done and 
            the widgets will automatically rotate to show the most relevant information throughout the day you can also swipe through them if you want to switch to a different one select multiple photos at the same time and then add them to other apps with this multi-touch gestures touch and hold a photo until you feel attacked and then dragged it away a little while still holding it with a different finger tap another photo to add it to the stack 
            you can select as many photos as you like once you selected the photos switch to the supported app you want to share two while still holding the Stock drag your finger wherewant to add them and lift your finger to drop them into the app if you find yourself typing out the same phrase a lot creative text replacement to get the message across faster in settings tap General then tap keyboard and tap text replacement tap the add button in the top-right corner and enter a phrase into the phrase field then enter the text replacement you like to use into the shortcut field and tap save now whenever you type the text replacement the phrase will appear in the predictive text section of your keyboard you can quickly swipe to open the camera without even having to unlock your screen just swipe left got itdid you know that you can scan a document with a Notes app on your iPhone in a note tap the camera button and tap scan documents position the document in the viewfinder and it will scan automatically repeat the process with his men Pages as you need to scan tap save when you're done starting an iOS 15 you can use live text to interact with text using your camera just position of you find her over a document and tap the scan text button in the bottom right corner you can now interact with a text that appears for example you can tap select all then tap the screen to see the editing options again and tap copy then switch over to another app and paste the text that you just copiedin the photos app if you're looking for the first 
            photo in an album library or search results you can quickly jump to the beginning of the view by tapping the top edge of your iPhone screen tap the tab button for the view you're in at the bottom of the screen to jump back to the most recent photo you can quickly access search right on your home screen and an iOS 15 your lock screen to just wipe your finger down from the middle of the home screen or lock screen and a search field will appear type in what you're looking for to quickly launch an app find a contact get web search suggestions and more gouda mac and cheese sounds delicious with these helpful tips here on your way to discovering more of what your iPhone has to offer to learn more about how to use your iPhone subscribe to the Apple support YouTube channelgo to keep watching Wikipedia is a registered trademark of the Wikimedia Foundation Incorporated a nonprofit organization
         """
#<텍스트 문장화_nltk 이용>
#sent_doc = maksen_v(dup_doc)    #동사 기준 구분
sent_doc = maksen_cc(dup_doc)    #연결사 기준 구분
  
#<semantic search를 위한 검색어 입력>
query = str(input("검색어를 입력해주세요 : "))
  
#<semantic search 과정_faiss와 SBERT이용 함수>
result = semantic(sent_doc, query)

#<결과 출력_faiss와 SBERT이용 함수>  
print('<results>')
for res in result:
    print('\t',res)



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
검색어를 입력해주세요 : taking photo
<results>
	 and hold the item you want to pin
	 and hold a photo until you feel attacked
	 back to the most recent photo you can
	 and then dragged it away a little while
	 and enter a phrase into the phrase field
